## Working with Key/Value Pairs (ch 4)

Spark provides special operations on RDDs containing key/value pairs. 

These RDDs are called *pair RDDs*. Pair RDDs are a useful building block in many programs, as they expose operations that allow you to act on each key in parallel or regroup data across the network. For example, pair RDDs have a **reduceByKey()** method that can aggregate data separately for each key, and a **join()** method that can merge two RDDs together by grouping elements with the same key. 

It is common to extract fields from an RDD (representing, for instance, an event time, customer ID, or other identifier) and use those fields as keys in pair RDD operations.

In [0]:
lines = sc.textFile('/FileStore/tables/Dante_Inferno.txt')
lines = lines.map(lambda s: s.replace('\x92',''))

In [0]:
lines.count()

create a set of `<key,value>` pairs where the key is the length of the line

In [0]:
linesWithLength = lines.map(lambda x: (len(x), x))
linesWithLength

In [0]:
linesWithLength.take(10)

In [0]:
linesWithLength.sortByKey(ascending=False).take(10)

## exercise: compute the average length of the lines using the keys -- without built in stats functions

In [0]:
sumLength = linesWithLength.keys().reduce(lambda x,y: x+y)

In [0]:
sumLength

In [0]:
avgLength = sumLength / float(linesWithLength.count())
avgLength

select lines that are shorter than the average length

In [0]:
shortLines = linesWithLength.filter(lambda pair: pair[0] < avgLength)

In [0]:
linesWithLength.count(),shortLines.count()

The canonical MapReduce **word count example**: computes <word, count> for each word in the document

Split each line into words

In [0]:
words = lines.flatMap(lambda l: l.split(" "))

filter out noise (optional step)

In [0]:
filteredWords = words.filter(lambda x:  x != ''  and not x.startswith('\\') and len(x) > 5)

count words

In [0]:
wordCount = filteredWords.map(lambda x: (x,1)).reduceByKey(lambda x, y: x+y)

In [0]:
wordCount.take(10)

#### Some simple analytics on the wordCount RDD: list the top-10 most frequent words

note that the pairs are not sorted by key, so we need to flip the pairs first, creating frequencyWordPairs

In [0]:
wordCount.sortBy(lambda x: x[1], ascending=False).take(5)

In [0]:
frequencyWordPairs = wordCount.map(lambda x: (x[1],x[0]))
frequencyWordPairs.take(10)

In [0]:
frequencyWordPairs.sortByKey(ascending=False).take(10)

### exercise: group words into sets, words in each set have the same frequency: <frequency, [words]>

we introduce a new grouping function:  **groupByKey()**

In [0]:
sameFrequencyGroups = frequencyWordPairs.groupByKey()
sameFrequencyGroups

In [0]:
sameFrequencyGroups.sortByKey(ascending=False)

In [0]:
sameFrequencyGroups.take(3)

suppose we are only interested in groups with more than one element

In [0]:
nonSingletonGroups = sameFrequencyGroups.filter(lambda x: len(x[1])>1)

In [0]:
for (f,l) in nonSingletonGroups.take(7):
    print("words with frequency ",f)
    for w in l:
        print(w)

**Exercise**: create a set of words for each frequency value, using **combineByKey()** in combination with the frequencyWordPairs RDD

### Joins

Example: (id, age) join (id, salary) using id as key:

In [0]:
idAge = sc.parallelize([(1,20), (2,25), (3,40), (4,45), (3,50)])
idIncome = sc.parallelize([(1,10000), (2,30000), (3,40000)])

1. full join. note that this leaves out id=4  (why??)

In [0]:
idAge.join(idIncome).collect()

outer join -- includes id = 4

In [0]:
idAge.leftOuterJoin(idIncome).collect()